In [ ]:
!pip uninstall -y imageio imageio-ffmpeg
!pip install imageio[ffmpeg]


Found existing installation: imageio 2.37.0
Uninstalling imageio-2.37.0:
  Successfully uninstalled imageio-2.37.0
Found existing installation: imageio-ffmpeg 0.6.0
Uninstalling imageio-ffmpeg-0.6.0:
  Successfully uninstalled imageio-ffmpeg-0.6.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 33.7 MB/s eta 0:00:00


In [ ]:
!python3 -c "import imageio; print(imageio.plugins)"

<module 'imageio.plugins' from '/usr/local/lib/python3.11/dist-packages/imageio/plugins/__init__.py'>


In [ ]:
import imageio
imageio.plugins.ffmpeg  # Ensure ffmpeg plugin is loaded


<module 'imageio.plugins.ffmpeg' from '/usr/local/lib/python3.11/dist-packages/imageio/plugins/ffmpeg.py'>

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

--2025-06-17 12:52:56--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.6.1/cloudflared-linux-amd64.deb [following]
--2025-06-17 12:52:56--  https://github.com/cloudflare/cloudflared/releases/download/2025.6.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/f2a50f07-6f2f-47cd-8a16-5d3dca82800f?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250617%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250617T125256Z&X-Amz-Expires=300&X-Amz-Signature=185f3af5a766022c873e60c9c3250ca781809092e514cbac83800005c4b7

In [ ]:
%%writefile main.py
import subprocess
import multiprocessing
import socket
import re
import time
import urllib.request
from flask import Flask, request, jsonify, send_file
from diffusers import StableVideoDiffusionPipeline
import torch
import tempfile
import imageio
import imageio.plugins.ffmpeg
from PIL import Image
import numpy as np

port = 5000
app = Flask(__name__)
pipe = None

def wait_for_flask():
    print(f"⏳ Waiting for Flask to start on port {port}...")
    while True:
        try:
            with socket.create_connection(("127.0.0.1", port), timeout=1):
                print("✅ Flask is running locally.")
                break
        except OSError:
            time.sleep(1)

def start_cloudflared():
    wait_for_flask()
    print("🌐 Launching Cloudflare Tunnel...")
    process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
    )

    for line in iter(process.stdout.readline, b''):
        decoded = line.decode().strip()
        print("🌐 [cloudflared]:", decoded)  # See all output
        match = re.search(r"https://.*trycloudflare.com", decoded)
        if match:
            print("🔗 Access ComfyUI here:", match.group())
            break

@app.route("/i2v", methods=["POST"])
def i2v():
    try:
        if not request.is_json:
            return jsonify({'error': 'Request must be JSON'}), 400

        img_data = request.json.get("image")
        if img_data is None:
            return jsonify({'error': 'No image data provided'}), 400

        # Reconstruct NumPy array
        img_np = np.array(img_data, dtype=np.uint8)
        if img_np.ndim != 3 or img_np.shape[2] != 3:
            return jsonify({'error': 'Invalid image shape'}), 400

        # Convert to PIL Image
        image = Image.fromarray(img_np).convert("RGB")

        # Run inference
        video_frames = pipe(image, num_frames=8, decode_chunk_size=1, num_inference_steps=20).frames

        frames = []
        for f in video_frames:
            frames.extend(f if isinstance(f, list) else [f])

        out_frames = []
        for frame in frames:
            if isinstance(frame, Image.Image):
                frame_np = np.array(frame.convert("RGB"))
            elif isinstance(frame, np.ndarray):
                frame_np = frame
            else:
                return jsonify({'error': f'Unsupported frame type: {type(frame)}'}), 500

            out_frames.append(frame_np.tolist())

        with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as tmpfile:
            video_path = tmpfile.name
            imageio.mimwrite(video_path, out_frames, fps=4, format="mp4")
            return send_file(video_path, mimetype="video/mp4")

        # return jsonify({"video": video_base64})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

def start_flask():
    global pipe
    pipe = StableVideoDiffusionPipeline.from_pretrained(
        "stabilityai/stable-video-diffusion-img2vid",
        torch_dtype=torch.float16,
        variant="fp16"
    ).to("cuda")
    app.run(host="0.0.0.0", port=port)

def main():
    multiprocessing.set_start_method("spawn", force=True)

    flask_proc = multiprocessing.Process(target=start_flask)
    tunnel_proc = multiprocessing.Process(target=start_cloudflared)

    flask_proc.start()
    tunnel_proc.start()

    try:
        while flask_proc.is_alive():
            print("🟢 Flask is running... keeping process alive.")
            time.sleep(60)
    except KeyboardInterrupt:
        print("🛑 Stopping...")
        flask_proc.terminate()
        tunnel_proc.terminate()

    flask_proc.join()
    tunnel_proc.join()

if __name__ == "__main__":
    main()


Overwriting main.py


In [ ]:
!python3 main.py

2025-06-17 13:26:48.953642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750166808.995755   14429 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750166809.005672   14429 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 13:26:49.038226: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
🟢 Flask is running... keeping process alive.
2025-06-17 13:27:00.783443: E external/local_xla/xla/stream_executor/cud